В этом ноутбуке обучим модель MobileBERT распознавать спам в SMS

In [1]:
model_name = 'mobilebert'
train_dataset_name = 'spam sms'

In [2]:
!pip install tensorflow-text
import tensorflow_text as text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

Загружаем данные

In [4]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/data_for_colab/spam_sms.csv', encoding = "ISO-8859-1")
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [5]:
df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)
df.columns = ['IS_SPAM', 'DATA_COLUMN']
df['IS_SPAM'] = (df['IS_SPAM'] == 'spam').astype(int)

In [6]:
df

,IS_SPAM,DATA_COLUMN
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [7]:
df['IS_SPAM'].value_counts()

0    4825
1     747
Name: IS_SPAM, dtype: int64

In [8]:
df_positive = df[df['IS_SPAM']==1]

In [9]:
df_negative = df[df['IS_SPAM']==0]

Создаем тестовую и обучающую выборки

In [10]:
# Тестовая выборка
n_test = df_negative.shape[0] // 2
df_negative_test = df_negative.tail(n_test)
n_test = df_positive.shape[0] // 2
df_positive_test = df_positive.tail(n_test)

In [11]:
df_negative_test.shape

(2412, 2)

In [12]:
df_positive_test.shape

(373, 2)

In [13]:
df_positive_test

,IS_SPAM,DATA_COLUMN
2728,1,Urgent Please call 09066612661 from landline. ...
2729,1,Urgent! Please call 09066612661 from your land...
2741,1,I don't know u and u don't know me. Send CHAT ...
2766,1,Married local women looking for discreet actio...
2769,1,Burger King - Wanna play footy at a top stadiu...
...,...,...
5537,1,Want explicit SEX in 30 secs? Ring 02073162414...
5540,1,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
5547,1,Had your contract mobile 11 Mnths? Latest Moto...
5566,1,REMINDER FROM O2: To get 2.50 pounds free call...


In [14]:
df_balanced_test = pd.concat([df_negative_test, df_positive_test])

In [15]:
df_balanced_test.sample(10)

,IS_SPAM,DATA_COLUMN
4298,0,Hurt me... Tease me... Make me cry... But in t...
3793,0,Short But Cute: \Be a good person
4564,0,Honeybee Said: *I'm d Sweetest in d World* God...
4997,0,Can you talk with me..
2920,0,"Yo, any way we could pick something up tonight?"
4543,0,when you and derek done with class?
4688,0,"Didn't try, g and I decided not to head out"
4335,0,"Fyi I'm taking a quick shower, be at epsilon i..."
3355,0,Ok not a problem will get them a taxi. C ing ...
5386,0,I'm at work. Please call


In [16]:
df_balanced_test['IS_SPAM'].value_counts()

0    2412
1     373
Name: IS_SPAM, dtype: int64

In [17]:
# Обучающая выборка
n_train = df_negative.shape[0] // 2
df_negative_train = df_negative.head(n_train)
n_train = df_positive.shape[0] // 2
df_positive_train = df_positive.head(n_train)

In [18]:
df_balanced_train = pd.concat([df_negative_train, df_positive_train])

In [19]:
df_balanced_train['IS_SPAM'].value_counts()

0    2412
1     373
Name: IS_SPAM, dtype: int64

In [20]:
df_balanced_train.sample(10)

,IS_SPAM,DATA_COLUMN
1208,0,Also maaaan are you missing out
1313,0,How abt making some of the pics bigger?
117,0,Goodo! Yes we must speak friday - egg-potato r...
2655,0,Great! I have to run now so ttyl!
1619,0,Friends that u can stay on fb chat with
693,0,Will purchase d stuff today and mail to you. D...
1403,0,cool. We will have fun practicing making babies!
2632,0,I WILL CAL YOU SIR. In meeting
628,1,New TEXTBUDDY Chat 2 horny guys in ur area 4 j...
1870,0,"Aight, I'll text you when I'm back"


In [21]:
X_train = df_balanced_train['DATA_COLUMN'].squeeze()
y_train = df_balanced_train['IS_SPAM'].squeeze()

In [22]:
X_test = df_balanced_test['DATA_COLUMN'].squeeze()
y_test = df_balanced_test['IS_SPAM'].squeeze()

Работаем с моделью

In [23]:
mobilebert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3") # same as bert


In [24]:
mobilebert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/mobilebert_en_uncased_L-24_H-128_B-512_A-4_F-4_OPT/1", trainable=True)

In [25]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = mobilebert_preprocess(text_input)
outputs = mobilebert_encoder(preprocessed_text)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [26]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

In [27]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [28]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [29]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

In [30]:
history = model.fit(X_train, y_train, epochs=15)

Epoch 1/15
88/88 [==============================] - 242s 492ms/step - loss: 26153.8926 - accuracy: 0.8801 - precision: 0.5396 - recall: 0.7131
Epoch 2/15
88/88 [==============================] - 35s 396ms/step - loss: 0.1352 - accuracy: 0.9724 - precision: 0.8978 - recall: 0.8954
Epoch 3/15
88/88 [==============================] - 33s 381ms/step - loss: 0.0821 - accuracy: 0.9828 - precision: 0.9404 - recall: 0.9303
Epoch 4/15
88/88 [==============================] - 32s 361ms/step - loss: 0.0517 - accuracy: 0.9860 - precision: 0.9563 - recall: 0.9383
Epoch 5/15
88/88 [==============================] - 31s 358ms/step - loss: 0.0468 - accuracy: 0.9874 - precision: 0.9568 - recall: 0.9491
Epoch 6/15
88/88 [==============================] - 31s 352ms/step - loss: 0.0365 - accuracy: 0.9914 - precision: 0.9729 - recall: 0.9625
Epoch 7/15
88/88 [==============================] - 31s 348ms/step - loss: 0.0202 - accuracy: 0.9943 - precision: 0.9837 - recall: 0.9732
Epoch 8/15
88/88 [===========

In [31]:
y_predicted = model.predict(X_test)
y_predicted


88/88 [==============================] - 15s 144ms/step


array([[1.2645254e-06],
       [5.9022218e-09],
       [6.9544185e-05],
       ...,
       [9.9999964e-01],
       [9.9999988e-01],
       [9.9999988e-01]], dtype=float32)

In [32]:
y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [33]:
y_test

2795    0
2796    0
2797    0
2798    0
2799    0
       ..
5537    1
5540    1
5547    1
5566    1
5567    1
Name: IS_SPAM, Length: 2785, dtype: int64

In [34]:
accuracy_score(y_test, y_predicted)

0.9813285457809695

In [35]:
precision_score(y_test, y_predicted)


0.9104859335038363

In [36]:
recall_score(y_test, y_predicted)

0.9544235924932976

In [37]:
f1_score(y_test, y_predicted)

0.9319371727748692

In [38]:
df_results_on_test = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [39]:
index = model_name + ' trained on ' + train_dataset_name + ' and tested on ' + train_dataset_name + ' dataset'
df_results_on_test.loc[index, 'accuracy'] = accuracy_score(y_test, y_predicted)
df_results_on_test.loc[index, 'precision'] = precision_score(y_test, y_predicted)
df_results_on_test.loc[index, 'recall'] = recall_score(y_test, y_predicted)
df_results_on_test.loc[index, 'f1_score'] =  f1_score(y_test, y_predicted)

In [40]:
df_results_on_test

,accuracy,precision,recall,f1_score
mobilebert trained on spam sms and tested on spam sms dataset,0.981329,0.910486,0.954424,0.931937


Сохраним обученную модель

In [41]:
saved_model_path = '/content/drive/MyDrive/data_for_colab/mobilebert_trained_on_spam_sms_20_january'

In [42]:
print(saved_model_path)

/content/drive/MyDrive/data_for_colab/mobilebert_trained_on_spam_sms_20_january


In [43]:
model.save(saved_model_path, include_optimizer=True) 

Сохраним датафреймы с результатами на обучающей выборке и на тестовой выборке

Сначала результаты для обучающей выборки

In [44]:
name_for_train_csv = model_name + ' trained on ' + train_dataset_name + ' quality on train dataset'

In [45]:
def calculate_f1_score(precision, recall):
    F1 = 2 * (precision * recall) / (precision + recall)
    return F1

In [46]:
df_with_train_quality = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [47]:
for el in ['accuracy', 'precision', 'recall']:
    df_with_train_quality.loc[name_for_train_csv, el] = history.history[el][-1]
df_with_train_quality.loc[name_for_train_csv, 'f1_score'] = calculate_f1_score(history.history['precision'][-1], history.history['recall'][-1])

In [48]:
df_with_train_quality

,accuracy,precision,recall,f1_score
mobilebert trained on spam sms quality on train dataset,0.997487,0.991935,0.989276,0.990604


In [49]:
df_with_train_quality.to_csv('/content/drive/MyDrive/data_for_colab/dataframes/train_quality/' + name_for_train_csv + '.csv')

Теперь результаты для тестовой выборки

In [50]:
name_for_test_csv = model_name + ' trained on ' + train_dataset_name + ' and tested on ' + train_dataset_name + ' dataset'

In [51]:
df_results_on_test.to_csv('/content/drive/MyDrive/data_for_colab/dataframes/test_quality/' + name_for_test_csv + '.csv')